In [3]:
import pandas as pd
import google.generativeai as genai
import os
from dotenv import load_dotenv
from transformers import pipeline

# データの読み込み

In [5]:
df = pd.read_excel("Day1_アンケート .xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2996 entries, 0 to 2995
Data columns (total 22 columns):
 #   Column                                                                  Non-Null Count  Dtype 
---  ------                                                                  --------------  ----- 
 0   申込区分                                                                    2996 non-null   object
 1   申込区分（日本語）                                                               2996 non-null   object
 2   ロール                                                                     2996 non-null   object
 3   本日の総合的な満足度を５段階で教えてください。                                                 2996 non-null   int64 
 4   本日の講義内容について５段階で教えてください。 
学習量は適切だった                                      2996 non-null   int64 
 5   本日の講義内容について５段階で教えてください。 
講義内容が十分に理解できた                                  2996 non-null   int64 
 6   本日の講義内容について５段階で教えてください。 
運営側のアナウンスが適切だった                                2996 non-null   

In [5]:
df.columns

Index(['申込区分', '申込区分（日本語）', 'ロール', '本日の総合的な満足度を５段階で教えてください。 ',
       '本日の講義内容について５段階で教えてください。 \n学習量は適切だった',
       '本日の講義内容について５段階で教えてください。 \n講義内容が十分に理解できた',
       '本日の講義内容について５段階で教えてください。 \n運営側のアナウンスが適切だった',
       '本日の講師の総合的な満足度を５段階で教えてください。', '本日の講師について５段階で教えてください。\n授業時間を効率的に使っていた',
       '本日の講師について５段階で教えてください。\n質問に丁寧に対応してくれた',
       '本日の講師について５段階で教えてください。\n話し方や声の大きさが適切だった',
       'ご自身について５段階で教えてください。\n事前に予習をした', 'ご自身について５段階で教えてください。\n意欲をもって講義に臨んだ',
       'ご自身について５段階で教えてください。\n今回学んだことを学習や研究に生かせる', '親しいご友人にこの講義の受講をお薦めしますか？',
       '【必須】本日の講義で学んだことを50文字以上で入力してください。',
       '（任意）本日の講義で特によかった部分について、具体的にお教えください。',
       '（任意）分かりにくかった部分や改善点などがあれば、具体的にお教えください。',
       '（任意）講師について、よかった点や不満があった点などについて、具体的にお教えください。',
       '（任意）今後開講してほしい講義・分野などがあればお書きください。', '（任意）ご自由にご意見をお書きください。',
       '(任意) 今回がテスト運用となるチャットbotですが、今後改善をしていきたいと思います。気になる点、改善要望等あればぜひFBをお願いします。'],
      dtype='object')

In [25]:
df.columns = range(1,df.shape[1]+1)
df.head()

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,22
0,OTHER,不明,student,2,3,3,2,2,4,2,...,3,1,5,並列で学習させる際の種類について,半導体の話がおもしろそうだと思えたこと,OmniCampusにZoomのリンクを載せる動線では駄目でしょうか？_毎回、動線が散漫とし...,NaN,松尾研がどのような研究をされているのか知りたいです。,昨年資料にもありましたが、第１回目で各回の紹介は要らないように思います。,ぜひ英語表記を併記してほしいです！日本語表記になると訳文だとわかる日本語になってしまい、本当...
1,OTHER,不明,student,2,2,2,2,1,1,3,...,4,3,3,主に大規模言語モデル(LLM)の概要や講義の前半について学びました。LLMは単語列の生成確率...,NaN,NaN,NaN,NaN,NaN,NaN
2,OTHER,不明,student,2,3,3,3,3,4,3,...,4,2,5,大規模言語モデルの概要、今後学んでいくことの概要を認識できた。,今後の講義を外観できた。,NaN,NaN,NaN,初学者のため、理解は薄いと思いますが、今後の講義の全体感をつかめました。引き続き、よろしくお...,NaN
3,EDUCATION,学生,student,2,4,4,2,2,3,4,...,4,5,7,LLMの大まかな構造とこれからやっていくこと、LLM が世間ではどのように実装され実践的に活...,NaN,NaN,NaN,NaN,NaN,NaN
4,SPONSOR,会員企業,student,2,2,3,3,3,3,3,...,2,2,2,・講座概要\n・LLMの概要\n └次の文字列を確率的に計算しているモデルである\n └これ...,A先生の講義,各回の概要について。全体像を掴めたのは良かったが、1時間超かけてやるべきかは議論の余地がある...,NaN,見逃しているだけだと思うが、講義内容は知識としての内容が多い印象なので、実際のアプリケーショ...,NaN,NaN


In [36]:
positive_columns = 17
negative_columns = 18
positive = df.loc[df[positive_columns].notnull(),positive_columns].to_list()
negative = df.loc[df[positive_columns].notnull(),negative_columns].to_list()

# geminiのAPIキーを用いる

In [50]:
api_key = os.environ.get("GOOGLE_API_KEY")

In [56]:
positive = positive[:100]

In [ ]:
import google.generativeai as genai
import pandas as pd
import json
import os

genai.configure(api_key=api_key) 
model = genai.GenerativeModel('gemini-1.5-flash-latest')

def classify_comments_batch_simple(comments: list[str] | pd.Series) -> dict:
    """
    複数のコメントをバッチ処理で感情分析し、
    結果をシンプルなJSONオブジェクト（Pythonでは辞書）で返す関数。
    """
    if not isinstance(comments, list):
        comments = comments.to_list()

    if not comments:
        print("分析対象のコメントがありません。")
        return None
    
    # プロンプトのテンプレート
    prompt_template = """
あなたは、講義アンケートのコメントを分析する専門のアナリストです。
これからJSON形式で渡されるコメントのリストを分析し、以下のルールに従って結果を一つのJSONオブジェクトとして出力してください。

# 全体のルール
- あなたの応答は、解説や前置きなしに、指定されたJSONフォーマットのコードブロックのみにしてください。
- 入力されたコメントのインデックス番号は0から始まります。

# 分類ルール
各コメントに対して、以下の2つの分類を行ってください。
1.  **sentiment**: コメントの感情を分析し、以下のいずれかの数値を割り当ててください。
    - `0`: ポジティブ
    - `1`: ネガティブ
    - `2`: どちらでもない
2.  **topic**: コメントが何についての言及か、以下のカテゴリに分類してください。
    - `0`: 講義内容について
    - `1`: 講義資料について
    - `2`: 運営について
    - `3`: 講師について
    - `4`:チャットボットについて
    - `5`:その他

# 重要コメントの抽出ルール
コメント全体をレビューし、以下の基準に合致するコメントのインデックス番号（0から始まる番号）を特定し、該当するリストに追加してください。
- **frequent_topics**: 複数回（目安として3回以上）言及されている共通の意見や要望が含まれるコメントのインデックス。
- **urgent**: 緊急の対応が必要と思われるコメント（例：システム障害、重大な誤り）のインデックス。
- **suggestions**: 具体的な改善案が含まれているコメントのインデックス。

# 危険コメントの抽出ルール
コメント全体をレビューし、以下の基準に合致するコメントのインデックス番号を特定し、リストに追加してください。
- **dangerous**: 誹謗中傷、人格攻撃、違法行為を示唆するような不適切な内容を含むコメントのインデックス。

# 出力フォーマット
以下のJSON構造を厳守してください。各キーに対応する値は、インデックス番号の配列（リスト）です。

{
  "classifications": [
    {
      "comment_index": 0,
      "sentiment": 0,
      "topic": "1"
    },
    {
      "comment_index": 1,
      "sentiment": 1,
      "topic": "3"
    }
  ],
  "important_comments": {
    "frequent_topics": [5, 12, 28],
    "urgent": [15],
    "suggestions": [22]
  },
  "dangerous_comments": [45]
}

---
それでは、以下のコメントリストを分析してください。

【入力コメントリスト】
"""

    comment_list_str = json.dumps(comments, ensure_ascii=False)
    prompt = prompt_template + comment_list_str
    
    try:
        response = model.generate_content(prompt)
        clean_response = response.text.strip().replace('```json', '').replace('```', '').strip()
        analysis_result = json.loads(clean_response)
        return analysis_result
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        # print("APIからの応答:", response.text) # デバッグ用に残しておくと便利
        return None

# --- 実行例 ---
# サンプルデータを作成
sample_comments = [
    "説明が非常に分かりやすかった。", # 0
    "資料が見づらいです。文字を大きくしてほしい。", # 1
    "音声が何度も途切れました。至急対応してください。", # 2
    "素晴らしい講義をありがとうございました。", # 3
    "演習の時間が足りないと感じました。", # 4
    "あの講師は本当に何も知らない。最悪だ。", # 5
    "講義の進むスピードが速すぎます。", # 6
    "もっと演習の時間を長くしてほしいです。" # 7
]

# このコードを実行するには、ご自身のAPIキーをセットしてください
results = classify_comments_batch_simple(positive)



print("--- 分析結果の例 ---")
print(json.dumps(results, indent=2, ensure_ascii=False))


--- 分析結果の例 ---
{
  "classifications": [
    {
      "comment_index": 0,
      "sentiment": 0,
      "topic": 0
    },
    {
      "comment_index": 1,
      "sentiment": 0,
      "topic": 0
    },
    {
      "comment_index": 2,
      "sentiment": 2,
      "topic": 3
    },
    {
      "comment_index": 3,
      "sentiment": 0,
      "topic": 0
    },
    {
      "comment_index": 4,
      "sentiment": 0,
      "topic": 0
    },
    {
      "comment_index": 5,
      "sentiment": 0,
      "topic": 0
    },
    {
      "comment_index": 6,
      "sentiment": 0,
      "topic": 0
    },
    {
      "comment_index": 7,
      "sentiment": 0,
      "topic": 0
    },
    {
      "comment_index": 8,
      "sentiment": 0,
      "topic": 0
    },
    {
      "comment_index": 9,
      "sentiment": 0,
      "topic": 1
    },
    {
      "comment_index": 10,
      "sentiment": 0,
      "topic": 3
    },
    {
      "comment_index": 11,
      "sentiment": 0,
      "topic": 0
    },
    {
      "comment_i